In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from datetime import datetime
import cartopy.crs as ccrs
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patheffects as PathEffects
from pathlib import Path
import xarray as xr
# local modules
import mypaths
from common_defs import FLIGHTS, MASIN_FILE_MASK
import sat_tools
from cart import ukmo_igp_map

In [3]:
flight_date = datetime(2018, 3, 9, 11)
flight_id = [k for k, v in FLIGHTS.items() if v==f'{flight_date:%Y%m%d}'][0]

In [4]:
sat_opt = dict(instrument='avhrr', platform='metopb', channel='band2_vis')

In [5]:
# %debug
url = sat_tools.get_nearest_url(flight_date, **sat_opt)

In [6]:
sat_image_name = mypaths.sat_dir / f'{flight_date:%Y%m%d}' / Path(url).name
if not sat_image_name.is_file():
    stat = sat_tools.download_file(url=url, save_dir=mypaths.sat_dir/f'{flight_date:%Y%m%d}')
    assert stat == 0

In [7]:
im, extent, crs = sat_tools.read_raster_stereographic(str(sat_image_name))

In [8]:
projection = ccrs.Stereographic(central_latitude=68.5, central_longitude=-21)
# xticks = list(np.arange(-180, 180, 5))
# yticks = list(np.arange(0, 90, 1))
mapkw = dict(transform=ccrs.PlateCarree())

In [9]:
gridline_kw = dict(linestyle=(0, (10, 10)), linewidth=0.5, color='C9')

In [10]:
stride = 10

In [11]:
masin_data_path = mypaths.masin_dir / f'flight{flight_id}' / MASIN_FILE_MASK.format(flight_date=flight_date,
                                                                                    flight_id=flight_id)

In [12]:
ds = xr.open_dataset(masin_data_path, decode_times=False)

In [13]:
x = ds.LON_OXTS[~ds.ALT_OXTS.isnull()].values#[8000:]
y = ds.LAT_OXTS[~ds.ALT_OXTS.isnull()].values#[8000:]
z = ds.ALT_OXTS[~ds.ALT_OXTS.isnull()].values#[8000:]

In [17]:
cmap = plt.cm.plasma

In [ ]:
fig = plt.figure(figsize=(15, 7))
ax = ukmo_igp_map(fig, coast=dict(scale='50m', facecolor='none', edgecolor='C8'), **gridline_kw)

h = ax.imshow(im[::stride, ::stride], origin='upper', extent=extent, transform=crs, cmap='gray', interpolation='nearest')

points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

lc = LineCollection(segments, cmap=cmap, linewidth=3, path_effects=[
        PathEffects.withStroke(linewidth=5, foreground="w")],
                    norm=plt.Normalize(np.nanmin(z), np.nanmax(z)), **mapkw)
lc.set_array(z)
h = ax.add_collection(lc)
cb = fig.colorbar(h, ax=ax, shrink=0.9)
cb.ax.tick_params(labelsize=20)
cb.ax.set_title('Altitude (ft)');

In [16]:
fig.savefig(mypaths.plotdir / f'flight_{flight_id}_{"_".join(sat_opt.values())}.png', dpi=100, bbox_inches='tight')